# Tutorial 2 - A simple pipeline with signal postprocessing.

Here we demonstrate how to build a *MathAgent* as an intermediate to process the 
*SineGeneratorAgent*'s output before plotting.
Subsequently, a *MultiMathAgent* is built to show the ability to send a dictionary of 
multiple fields to the recipient. We provide a custom `on_received_message()` function, 
which is called every time a message is received from input agents.

The received message is a dictionary of the form:

    {
    'from':agent_name,
    'data': data,
    'senderType': agent_class_name,
    'channel':'channel_name'
    }

By default, `'channel'` is set to `"default"`, however a custom channel can be set 
when needed, which is demonstrated in the next tutorial.

In [4]:
# %load tutorial_2_math_agent.py
from agentMET4FOF.agents import AgentMET4FOF, AgentNetwork, MonitorAgent
from agentMET4FOF.streams import SineGenerator


class MathAgent(AgentMET4FOF):
    def on_received_message(self, message):
        data = self.divide_by_two(message["data"])
        self.send_output(data)

    # Define simple math functions.
    @staticmethod
    def divide_by_two(numerator: float) -> float:
        return numerator / 2


class MultiMathAgent(AgentMET4FOF):

    _minus_param: float
    _plus_param: float

    def init_parameters(self, minus_param=0.5, plus_param=0.5):
        self._minus_param = minus_param
        self._plus_param = plus_param

    def on_received_message(self, message):
        minus_data = self.minus(message["data"], self._minus_param)
        plus_data = self.plus(message["data"], self._plus_param)

        self.send_output({"minus": minus_data, "plus": plus_data})

    @staticmethod
    def minus(minuend: float, subtrahend: float) -> float:
        return minuend - subtrahend

    @staticmethod
    def plus(summand_1: float, summand_2: float) -> float:
        return summand_1 + summand_2


class SineGeneratorAgent(AgentMET4FOF):

    _stream: SineGenerator

    def init_parameters(self):
        self._stream = SineGenerator()

    def agent_loop(self):
        if self.current_state == "Running":
            sine_data = self._stream.next_sample()  # dictionary
            self.send_output(sine_data["x"])


def main():
    # start agent network server
    agentNetwork = AgentNetwork()
    # init agents
    gen_agent = agentNetwork.add_agent(agentType=SineGeneratorAgent)
    math_agent = agentNetwork.add_agent(agentType=MathAgent)
    multi_math_agent = agentNetwork.add_agent(agentType=MultiMathAgent)
    monitor_agent = agentNetwork.add_agent(agentType=MonitorAgent)
    # connect agents : We can connect multiple agents to any particular agent
    agentNetwork.bind_agents(gen_agent, math_agent)
    agentNetwork.bind_agents(gen_agent, multi_math_agent)
    # connect
    agentNetwork.bind_agents(gen_agent, monitor_agent)
    agentNetwork.bind_agents(math_agent, monitor_agent)
    agentNetwork.bind_agents(multi_math_agent, monitor_agent)
    # set all agents states to "Running"
    agentNetwork.set_running_state()

    # allow for shutting down the network after execution
    return agentNetwork


if __name__ == "__main__":
    main()


Starting NameServer...
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:3333 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:3333
INFO [2020-07-08 20:12:37.967786] (AgentController): INITIALIZED
Dash is running on http://127.0.0.1:8050/

INFO [2020-07-08 20:12:38.108067] (SineGeneratorAgent_1): INITIALIZED
 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "agentMET4FOF.dashboard.Dashboard" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
INFO [2020-07-08 20:12:38.185234] (MathAgent_1): INITIALIZED
INFO [2020-07-08 20:12:38.266143] (MultiMathAgent_1): INITIALIZED
INFO [2020-07-08 20:12:38.347055] (MonitorAgent_1): INITIALIZED
[2020-07-08 20:12:38.413035] (SineGeneratorAgent_1): Connected output module: MathAgent_1
[2020-07-08 20:12:38.418991] (SineGeneratorAgent_1): Connected output module: MultiMathAgent_1
[2020-07-08 20:12:38.425940] (SineGeneratorAgent_1): Connected output mo

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Jul/2020 22:12:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2020 22:12:40] "POST /_dash-update-component HTTP/1.1" 200 -
